In [2]:
import sqlite3

def get_table_names(file_path):
    
    # Create a SQL connection to our SQLite database
    con = sqlite3.connect(file_path)

    # creating cursor
    cur = con.cursor()

    # reading all table names
    table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

    # Be sure to close the connection
    con.close()

    table_names = [name[0] for name in table_list]

    return table_names


def print_data(file_path, table_name = None):
    
    if table_name is None:
        table_names = get_table_names(file_path)

        # Create a SQL connection to our SQLite database
        con = sqlite3.connect(file_path)

        cur = con.cursor()

        # The result of a "cursor.execute" can be iterated over by row
        for name in table_names:
            if name == 'ElementaryGeometries' or name == 'SpatialIndex':
                continue
            sql_command = "SELECT * FROM " + name
            print(name)
            for row in cur.execute(sql_command):
                print(row)
            print('\n\n')

        # Be sure to close the connection
        con.close()
    
    else:
        # Create a SQL connection to our SQLite database
        con = sqlite3.connect(file_path)

        cur = con.cursor()

        # The result of a "cursor.execute" can be iterated over by row
        sql_command = "SELECT * FROM " + table_name
        print(table_name)
        for row in cur.execute(sql_command):
            print(row)

        # Be sure to close the connection
        con.close()


def get_data(file_path, table_name = None):

    table_names = get_table_names(file_path)

    # Create a SQL connection to our SQLite database
    con = sqlite3.connect(file_path)

    cur = con.cursor()

    # The result of a "cursor.execute" can be iterated over by row
    for name in table_names:
        #if name == 'ElementaryGeometries' or name == 'SpatialIndex':
        #    continue
        sql_command = "SELECT * FROM " + name
        data_dict = {}
        for row in cur.execute(sql_command):
            print(row)
        print('\n\n')

    # Be sure to close the connection
    con.close()

In [ ]:
#trying git commit

In [11]:
import sqlite3

def generate_docs(db, docs, title):
    # open doc file and define table template
    f = open(docs, "w+")
    f.write(title + "\n")
    table_template = "|{}|{}|\n| :-: | :-:|\n".format("COLUMN", "DESCRIPTION")
    # open database
    con = sqlite3.connect(db)
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in cur.fetchall():
        # print table name
        table_name = table[0].upper()
        f.write("### {}\n".format(table_name))
        f.write(table_template)
        if table_name == 'ELEMENTARYGEOMETRIES' or table_name == 'SPATIALINDEX':
            continue
        # print table columns
        cur.execute("SELECT * FROM {};".format(table_name))
        cols = [description[0] for description in cur.description]
        for col in cols:
            f.write("| {} | |\n".format(col))
    f.close()


generate_docs("/Users/danielorzan/Desktop/ATIproject/DP08_XX_K01_C07_F2_N001.db", "db_template.md",
              "Room schema")

In [37]:
a = get_table_names('/Users/danielorzan/Desktop/ATIproject/DP08_XX_K01_C07_F2_N001.db')
print(a)

['__EFMigrationsHistory', 'spatial_ref_sys', 'spatialite_history', 'sqlite_sequence', 'geometry_columns', 'spatial_ref_sys_aux', 'views_geometry_columns', 'virts_geometry_columns', 'geometry_columns_statistics', 'views_geometry_columns_statistics', 'virts_geometry_columns_statistics', 'geometry_columns_field_infos', 'views_geometry_columns_field_infos', 'virts_geometry_columns_field_infos', 'geometry_columns_time', 'geometry_columns_auth', 'views_geometry_columns_auth', 'virts_geometry_columns_auth', 'sql_statements_log', 'SpatialIndex', 'ElementaryGeometries', 'Project', 'Document', 'DocumentParameter', 'Element', 'Workset', 'ElementParameterInteger', 'ElementParameterNumber', 'ElementParameterRevitId', 'ElementParameterText']


In [ ]:
print_data('/Users/danielorzan/Desktop/ATIproject/DP08_XX_K01_C07_F2_N001.db')

In [16]:
# issue with SpatialIndex and ElementaryGeometries
print_data('/Users/danielorzan/Desktop/ATIproject/DP08_XX_K01_C07_F2_N001.db', 'SpatialIndex')

SpatialIndex


OperationalError: no such module: VirtualSpatialIndex